In [1]:
import argparse
import logging
import time
from copy import deepcopy
from posixpath import split

import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from dn.data import (
    ContinousNWays,
    ImageData,
    ImageDataDS,
    MarketTaskset,
    MetaLoader,
    MyDS,
)
from dn.models import DualNetMarket
from dn.utils import deterministic, load_image_data_pickle

/home/harini/miniconda3/envs/DN/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/harini/miniconda3/envs/DN/lib/python3.7/site-packages/learn2learn/vision/benchmarks/omniglot_benchmark.py:7: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  from PIL.Image import LANCZOS


In [2]:
import sys

sys.argv = [""]
del sys


parser = argparse.ArgumentParser(description="DualNet-Image")
parser.add_argument("--batch_size", type=int, default=32)
parser.add_argument("--n_epochs", type=int, default=100)
parser.add_argument("--lr", type=float, default=0.001)
parser.add_argument("--ssl_lr", type=float, default=0.001)
parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--path", type=str, default="./data/dfcl.csv")
parser.add_argument("--hidden_dim", type=int, default=64)
parser.add_argument("--input_dim", type=int, default=28)
parser.add_argument("--out_dim", type=int, default=4)
parser.add_argument("--n_class", type=int, default=4)
parser.add_argument(
    "--n_memories", type=int, default=50, help="number of memories per task"
)
parser.add_argument(
    "--memory_strength",
    default=1.0,
    type=float,
    help="memory strength (meaning depends on memory)",
)
parser.add_argument("--reg", default=1.0, type=float)
parser.add_argument("--inner_steps", type=int, default=1)
parser.add_argument(
    "--temperature", type=float, default=1.0, help="temperature for distilation"
)
parser.add_argument("--beta", type=float, default=0.3)
parser.add_argument(
    "--save_path",
    type=str,
    default="results/",
    help="save models at the end of training",
)
parser.add_argument("--replay_batch_size", type=int, default=10)
parser.add_argument("--n_outer", type=int, default=1)
parser.add_argument("--device", type=str, default="cpu")
args = parser.parse_args()

In [3]:
writer = SummaryWriter(f"runs/test_2_{time.time()}")
deterministic(args)
device = torch.device(args.device)

train_taskset, test_taskset = MarketTaskset("data/dfcl_mini.csv"), MarketTaskset(
    "data/dfcl_mini.csv", split="Test"
)

model = DualNetMarket(args).to(device)
CLoss = torch.nn.CrossEntropyLoss()
KLLoss = torch.nn.KLDivLoss()

opt = torch.optim.SGD(model.parameters(), lr=args.lr)
ssl_opt = torch.optim.SGD(model.SlowLearner.parameters(), lr=args.ssl_lr)

AttributeError: 'Namespace' object has no attribute 'hidden_dim'